In [ ]:
# ollama installation from: https://medium.com/@abonia/running-ollama-in-google-colab-free-tier-545609258453
# faiss usage from: https://python.langchain.com/docs/integrations/vectorstores/faiss/

In [ ]:
!pip install colab-xterm
%load_ext colabxterm

In [ ]:
%xterm

In [ ]:
!pip install -qU langchain-ollama

In [ ]:
!pip install -qU langchain-community faiss-gpu

In [ ]:
!pip install bitsandbytes

In [ ]:
from langchain_community.vectorstores import FAISS
from langchain_ollama import OllamaEmbeddings

from peft import PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer

from google.colab import drive
import os

In [ ]:
os.environ['OLLAMA_HOST'] = '127.0.0.1:11434'

In [ ]:
drive.mount('/content/drive')

In [ ]:
embeddings = OllamaEmbeddings(model="llama3")

vector_store = FAISS.load_local(
    "/content/drive/MyDrive/index_faiss", embeddings, allow_dangerous_deserialization=True
)


In [ ]:
retriever = vector_store.as_retriever(search_type="similarity_score_threshold", search_kwargs={"k": 1, "score_threshold" : 0.2})

In [ ]:
finetuned_model = PeftModel.from_pretrained(AutoModelForCausalLM.from_pretrained('facebook/opt-1.3b'), 'new_model')
baseline = AutoModelForCausalLM.from_pretrained('facebook/opt-1.3b')

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('facebook/opt-1.3b')

In [ ]:
question = "Where is Atlanta?"

text = question + retriever.invoke(question)[0].dict()['page_content']

print(retriever.invoke(question)[0].dict()['page_content'])
tokenized = tokenizer(text, return_tensors='pt', padding=True)
output =  finetuned_model.generate(tokenized.input_ids, max_new_tokens = 100) # Can update with either baseline or finetuned_model
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print(generated_text)